# [ProjectEuler_648](https://projecteuler.net/problem=648)
For some fixed $\rho \in [0, 1]$, we begin a sum $s$ at $0$ and repeatedly apply a process:

- With probability $\rho$, we add $1$ to $s$, otherwise we add $2$ to $s$.

- The process ends when either $s$ is a perfect square or $s$ exceeds $10^{18}$, whichever occurs first.

For example, if $s$ goes through $0, 2, 3, 5, 7, 9$, the process ends at $s=9$, and two squares $1$ and $4$ were skipped over.

Let $f(\rho)$ be the expected number of perfect squares skipped over when the process finishes.
It can be shown that the power series for $f(\rho)$ is $\sum_{k=0}^\infty a_k \rho^k$ for a suitable (unique) choice of coefficients $a_k$. Some of the first few coefficients are $a_0=1$, $a_1=0$, $a_5=-18$, $a_{10}=45176$.
Let $F(n) = \sum_{k=0}^n a_k$. You are given that $F(10) = 53964$ and $F(50) \equiv 842418857 \pmod{10^9}$.
Find $F(1000)$, and give your answer modulo $10^9$.

## Theoretical findings
Let us consider a random process $X_n$ such that it mimics the process $s$ in the problem without stopping after hitting any square number or the upper bound $10^{18}$. We introduce
- $r_k = \mathbb{P}\left[ X_n  = k \text{ for some } n\right]$,
- $q_k = \mathbb{P}\left[ X_n  = k^2 + 1 \text{ for some } n \text{ and the process didn't hit any square number beforehand}\right]$.

As it is clear from the process definition, all probabilities $r_k, q_k$ can be written as polynomials of a single variable $\rho$. Then one can check that
- Not hitting the number $k$ by the process $X_n$ means that we are hitting number $k-1$ and then step over number $k$ with the step size $2$, which can be translated as
\begin{equation}
1-r_k = (1-\rho) r_{k-1}, \qquad k\geq 1.
\end{equation}
- Hitting the number $k^2 + 1$ without hitting any square beforehand is equivalent to first hitting the number $(k-1)^2+1$ without hitting a square number, then accumulating $k^2 - 1 - (k-1)^2 - 1 = 2k-3$ in total for some number of steps and then overstepping the number $k^2$ by making a step of size $2$. The above can be rewritten as
\begin{equation}
q_k = q_{k-1}*r_{2k-3}*(1-\rho), \qquad k\geq 2.
\end{equation}
- Finally, the average number of the squares skipped can be written as
\begin{equation}
f\left(\rho\right) = \sum\limits_{k=1}^{10^9} q_k.
\end{equation}

## Evaluation details
- From the above one can see that we are interested in $r_k$ for odd values of $k$ only and $\deg r_{2i+1} = 2i + 1, r_{2i+1} \vdots \rho$.
- It is also easy to spot that $\deg q_k = k^2 - k + 1, q_k \vdots \rho^{k-1}$ and we can cut our series at $k = N + 1$. We will then evaluate
\begin{equation}
\hat{f}\left(\rho\right) = \sum\limits_{k=1}^{N+1}q_k.
\end{equation}
- Therefore, we evaluate two sets of polynomials
\begin{equation}
\left\{ r_{2i+1}\right\}_{i = 0}^{N - 1} \text{ and }\left\{ q_{i+1}\right\}_{i,j = 0}^{N}.
\end{equation}

In [103]:
%reload_ext autoreload
%autoreload 2
import Polynomial as poly
import numpy as np
N = 10**3
M = 10**9

In [104]:
def solution(limit, modulo):
    rho_polynomial = poly.Polynomial([0, 1], limit, modulo)
    one_minus_rho_polynomial = poly.Polynomial([1, -1], limit, modulo)
    one_minus_rho_square_polynomial = poly.Polynomial([1, -2, 1], limit, modulo)

    r_k = [rho_polynomial]
    for _i in range(1, limit):
        r_k.append(rho_polynomial+ r_k[-1]*one_minus_rho_square_polynomial)

    q_k = [one_minus_rho_polynomial]
    for _i in range(1, limit+1):
        q_k.append(q_k[-1]*one_minus_rho_polynomial*r_k[_i-1])

    f = np.sum([q_k[_i] for _i in range(limit+1)])
    return sum(f.coefficients) % modulo

## Testing

In [105]:
import unittest
class TestNotebook(unittest.TestCase):
    def test_solution_small(self):
        self.assertEqual(solution(10, M), 53964)

    def test_solution_medium(self):
        self.assertEqual(solution(50, M), 842418857)

unittest.main(argv = [''], verbosity = 2, exit = False)

test_solution_medium (__main__.TestNotebook.test_solution_medium) ... ok
test_solution_small (__main__.TestNotebook.test_solution_small) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.068s

OK


## Final answer

In [106]:
solution(N, M)

301483197